Задача 1. 
Используя библиотеку SQLite.

In [1]:
import sqlite3

conn = sqlite3.connect('database.db')
cursor = conn.cursor()

sql1 = ''' 
        CREATE TABLE IF NOT EXISTS data (
            key INTEGER PRIMARY KEY,
            `Index` TEXT,
            Date DATE, 
            Open FLOAT, 
            High FLOAT, 
            Low FLOAT, 
            Close FLOAT, 
            "Adj close" FLOAT, 
            Volume INTEGER
        )
'''

cursor.execute(sql1)
print("data Table created successfully........")
conn.commit()

sql2 = ''' 
        CREATE TABLE IF NOT EXISTS info (
            Region text, 
            Exchange text, 
            "Index" text, 
            Currency text
        )
'''

cursor.execute(sql2)
print("info Table created successfully........")
conn.commit()


sql3 = ''' 
        CREATE TABLE IF NOT EXISTS processed (
            key INTEGER PRIMARY KEY,
            "Index" TEXT,
            Date DATE, 
            Open FLOAT, 
            High FLOAT, 
            Low FLOAT, 
            Close FLOAT, 
            "Adj close" FLOAT, 
            Volume INTEGER, 
            CloseUSD FLOAT
        )
'''

cursor.execute(sql3)
print("processed Table created successfully........")
conn.commit()
conn.close()

data Table created successfully........
info Table created successfully........
processed Table created successfully........


Задача 2. 
Я написал функцию на Python благодаря которому мы можем загрузить данные с csv-файла в таблицу в нашем базе данных.

In [2]:
import csv
def insert_values(db_name, table_name, csv_file, id=False):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    with open(csv_file, 'r') as f:
        reader = csv.reader(f)
        #ignore first line, which is name of columns
        cols = next(reader)
        if id:
            len_cols = len(cols) + 1
        else:
            len_cols = len(cols)
        query = "insert into {} values ({})".format(table_name, ','.join('?'*len_cols))
        for key, data in enumerate(reader,1):
            if id:
                cursor.execute(query, [key, *data])
            else:
                cursor.execute(query, data)
        conn.commit()
    conn.close()

insert_values('database.db', 'data', 'indexData.csv', id=True)
insert_values('database.db', 'info', 'indexInfo.csv', id=False)
insert_values('database.db', 'processed', 'indexProcessed.csv', id=True)

In [3]:
# Check tables
conn = sqlite3.connect('database.db')
cursor = conn.cursor()

for name in ['data', 'info', 'processed']:
    sql = 'select * from {} limit 5'.format(name)
    cursor.execute(sql)
    rows = cursor.fetchall()
    for row in rows:
        print(row)

(1, 'NYA', '1965-12-31', 528.690002, 528.690002, 528.690002, 528.690002, 528.690002, 0)
(2, 'NYA', '1966-01-03', 527.210022, 527.210022, 527.210022, 527.210022, 527.210022, 0)
(3, 'NYA', '1966-01-04', 527.840027, 527.840027, 527.840027, 527.840027, 527.840027, 0)
(4, 'NYA', '1966-01-05', 531.119995, 531.119995, 531.119995, 531.119995, 531.119995, 0)
(5, 'NYA', '1966-01-06', 532.070007, 532.070007, 532.070007, 532.070007, 532.070007, 0)
('United States', 'New York Stock Exchange', 'NYA', 'USD')
('United States', 'NASDAQ', 'IXIC', 'USD')
('Hong Kong', 'Hong Kong Stock Exchange', 'HSI', 'HKD')
('China', 'Shanghai Stock Exchange', '000001.SS', 'CNY')
('Japan', 'Tokyo Stock Exchange', 'N225', 'JPY')
(1, 'HSI', '1986-12-31', 2568.300049, 2568.300049, 2568.300049, 2568.300049, 2568.300049, 0, 333.87900637)
(2, 'HSI', '1987-01-02', 2540.100098, 2540.100098, 2540.100098, 2540.100098, 2540.100098, 0, 330.21301274)
(3, 'HSI', '1987-01-05', 2552.399902, 2552.399902, 2552.399902, 2552.399902, 2552.

Задача 3. 

In [4]:
# 1. V_NOTPROCESSED view
conn = sqlite3.connect('database.db')
cursor = conn.cursor()
sql = ''' 
    CREATE VIEW IF NOT EXISTS V_NOTPROCESSED
    AS 
    SELECT DISTINCT data.key, data.[Index] as "Index", data.Date, data.Open, data.High, data.Low, 
    data.Close, data.[Adj close], info.Region
    FROM data
    INNER JOIN processed ON processed.key = data.key
    INNER JOIN info ON info.[Index] = data.[Index]
    GROUP BY info.Region
'''
cursor.execute(sql)
conn.commit()

In [5]:
query = ''' 
    SELECT * FROM V_NOTPROCESSED
'''
cursor.execute(query)
rows = cursor.fetchall()
for row in rows:
    print(row)

(41352, 'GSPTSE', '1979-06-29', 1614.699951, 1619.800049, 1614.699951, 1618.400024, 1614.290039, 'Canada')
(35389, '000001.SS', '1997-07-02', 1255.909058, 1261.571045, 1147.331055, 1199.061035, 1199.061035, 'China')
(61437, 'GDAXI', '1987-12-30', 1005.190002, 1005.190002, 1005.190002, 1005.190002, 1005.190002, 'Germany')
(26639, 'HSI', '1986-12-31', 2568.300049, 2568.300049, 2568.300049, 2568.300049, 2568.300049, 'Hong Kong')
(58056, 'NSEI', '2007-09-17', 4518.450195, 4549.049805, 4482.850098, 4494.649902, 4494.649902, 'India')
(92451, 'N225', '1965-01-05', 1257.719971, 1257.719971, 1257.719971, 1257.719971, 1257.719971, 'Japan')
(70043, 'KS11', '1996-12-11', 705.98999, 709.47998, 704.429993, 704.679993, 704.679993, 'Korea')
(90064, 'J203.JO', '2012-02-08', 33974.44922, 34282.94141, 33974.44922, 34226.73047, 34226.73047, 'South Africa')
(76224, 'SSMI', '1990-11-09', 1378.900024, 1389.0, 1375.300049, 1387.099976, 1387.099976, 'Switzerland')
(84054, 'TWII', '1997-07-02', 9094.269531, 912

In [6]:
# 2. PROCESSED VIEW
sql = ''' 
    CREATE VIEW IF NOT EXISTS V_PROCESSED
    AS 
    SELECT processed.key, processed.[Index], processed.Date, processed.Open, 
    processed.High, processed.Low, processed.Close, processed.[Adj close], processed.Volume, 
    processed.CloseUSD, info.Currency, info.Exchange, MAX(processed.Open) AS MaxOpen, MIN(processed.Low) AS MinPrice, 
    strftime('%Y', processed.Date), strftime('%m', processed.Date), info.Region
    FROM processed
    INNER JOIN info ON processed.[Index] = info.[Index]
    GROUP BY info.Region, strftime('%Y', processed.Date), strftime('%m', processed.Date)
'''
cursor.execute(sql)
conn.commit()



In [7]:
query = ''' 
    SELECT * FROM V_PROCESSED
'''
cursor.execute(query)
rows = cursor.fetchall()
for row in rows:
    print(row)


(66029, 'GSPTSE', '1979-06-29', 1614.699951, 1619.800049, 1614.699951, 1618.400024, 1614.290039, 0, 1343.27201992, 'CAD', 'Toronto Stock Exchange', 1614.699951, 1614.699951, '1979', '06', 'Canada')
(66038, 'GSPTSE', '1979-07-13', 1533.699951, 1538.099976, 1520.599976, 1537.400024, 1533.49585, 0, 1276.04201992, 'CAD', 'Toronto Stock Exchange', 1613.599976, 1520.599976, '1979', '07', 'Canada')
(66055, 'GSPTSE', '1979-08-08', 1555.900024, 1567.099976, 1553.099976, 1565.199951, 1561.225098, 0, 1299.11595933, 'CAD', 'Toronto Stock Exchange', 1682.599976, 1553.099976, '1979', '08', 'Canada')
(66074, 'GSPTSE', '1979-09-05', 1681.0, 1681.0, 1659.400024, 1661.800049, 1657.579956, 0, 1379.29404067, 'CAD', 'Toronto Stock Exchange', 1758.800049, 1659.400024, '1979', '09', 'Canada')
(66106, 'GSPTSE', '1979-10-22', 1588.900024, 1588.900024, 1514.800049, 1537.300049, 1533.39624, 0, 1275.95904067, 'CAD', 'Toronto Stock Exchange', 1793.5, 1514.800049, '1979', '10', 'Canada')
(66119, 'GSPTSE', '1979-11-

In [30]:
#4. Визуализация используя pandas, matplotlib, ipywidgets
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import VBox, interact, Dropdown, Output
import numpy as np 

df = pd.DataFrame(rows)
# 16 - Region
# 15 - Months
# 12 - MaxOpen
# 13 - MinPrice
# 10 - Currency


df = df.groupby(by=[10, 15,16], as_index=False).agg(
    maxOpen = pd.NamedAgg(column = 12, aggfunc = 'max'),
    minPrice = pd.NamedAgg(column = 13, aggfunc = 'min'))

regions = df[16].unique()

dropdown1 = Dropdown(
    options=regions,
    value='Canada',
    disabled=False, 
    description="Region 1: ")

dropdown2 = Dropdown(
    options=regions,
    value='Europe',
    disabled=False, 
    description="Region 2: ")

output = Output()

@interact(r1 = dropdown1, r2 = dropdown2)
def generate_plot(r1, r2):
    with output:
        output.clear_output()
        df1 = df[df[16] == r1]
        df2 = df[df[16]==r2]
        x = np.arange(12)
        width = 0.2

        plt.bar(x-0.4,df1.maxOpen, width, color="red")
        plt.bar(x-0.2, df1.minPrice, width, color="yellow")
        plt.bar(x, df2.maxOpen, width, color="blue")
        plt.bar(x + 0.2,df2.minPrice, width, color="purple")

        plt.xticks(x)
        plt.xlabel("Months")

        plt.legend(["MaxOpen of {}".format(r1), "MinPrice of {}".format(r1),
        "MaxOpen of {}".format(r2), "MinPrice of {}".format(r2)])
        plt.show()



VBox([output])

interactive(children=(Dropdown(description='Region 1: ', options=('Canada', 'Switzerland', 'China', 'Europe', …